1.7.1 Perfectly Matched Layers (PML)
====

### Source problem

In [1]:
import netgen.gui
from netgen.geom2d import SplineGeometry
from ngsolve import *

geo = SplineGeometry()
geo.AddCircle( (0,0), 1.4, leftdomain=2, bc="outerbnd")
geo.AddCircle( (0,0), 1, leftdomain=1, rightdomain=2, bc="innerbnd")
geo.SetMaterial(1, "inner")
geo.SetMaterial(2, "pml")
mesh = Mesh(geo.GenerateMesh (maxh=0.1))
mesh.Curve(3)
mesh.SetPML(pml.Radial(rad=1,alpha=0.2j,origin=(0,0)), "pml")

fes = H1(mesh, order=4, complex=True)
u = fes.TrialFunction()
v = fes.TestFunction()

omega = 10

a = BilinearForm(fes)
a += grad(u)*grad(v)*dx + (-1)* (omega*omega*u*v*dx) - 1j*omega*u*v*ds("outerbnd")

f = LinearForm(fes)
f += SymbolicLFI(exp(-20**2*((x-0.3)*(x-0.3)+y*y))*v)

a.Assemble()
f.Assemble()

gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse() * f.vec

Draw(gfu)

### Eigenvalue problem